In [1]:
push!(LOAD_PATH,".")

4-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "."

# Import CHAKRA (and an implementation)

In [2]:
using Chakra, ChakraImp

# Define an attribute type
The following defines a type pc of pitch classes and associates it an attribute name ":pc"

In [3]:
abstract type PC end
struct pc <: PC
    val::Char
end
Chakra.@associatedType(:pc,PC)

# Represent choral
The following represents the first 10 notes of a bach choral as a sequence of constituents with pitch class attributes.

In [4]:
e1 = set(delim(),:pc,pc('G'))
e2 = set(delim(),:pc,pc('G'))
e3 = set(delim(),:pc,pc('D'))
e4 = set(delim(),:pc,pc('B'))
e5 = set(delim(),:pc,pc('A'))
e6 = set(delim(),:pc,pc('G'))
e7 = set(delim(),:pc,pc('G'))
e8 = set(delim(),:pc,pc('A'))
e9 = set(delim(),:pc,pc('B'))
e10 = set(delim(),:pc,pc('A'))
choral = [e10,e9,e8,e7,e6,e5,e4,e3,e2,e1]

10-element Vector{ChakraImp.Obj}:
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('A')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('B')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('A')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('A')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('B')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('D')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))

# Define viewpoint
The following defines an atomic viewpoint which projects out the pitch class of the last constituent of a sequence

In [5]:
using Viewpoints
v = AtomicViewpoint(:pc)

AtomicViewpoint{PC}(:pc)

### vp_type gives the type of values of the viewpoint:

In [6]:
vp_type(v)

PC

### apply the viewpoint to a sequence as follows:

In [7]:
v(choral)

pc('A')

### vp_map turns a sequence of constituents in to a sequence of viewpoint values:

In [8]:
vp_map(v,choral)

10-element Vector{pc}:
 pc('A')
 pc('B')
 pc('A')
 pc('G')
 pc('G')
 pc('A')
 pc('B')
 pc('D')
 pc('G')
 pc('G')

### Viewpoints can be delayed, linked and derived by composition with arbitrary functions:

In [9]:
v2 = LinkedViewpoint(DelayedViewpoint(v,1),v)
vp_map(v2,choral)

9-element Vector{Tuple{pc, pc}}:
 (pc('B'), pc('A'))
 (pc('A'), pc('B'))
 (pc('G'), pc('A'))
 (pc('G'), pc('G'))
 (pc('A'), pc('G'))
 (pc('B'), pc('A'))
 (pc('D'), pc('B'))
 (pc('G'), pc('D'))
 (pc('G'), pc('G'))

# Create Model 
The following creates an HGramModel of the choral using the pitch class viewpoint. 

In [10]:
using Models
m = HGramModel(vp_map(v,choral),5)

┌ Info: Precompiling Models [top-level]
└ @ Base loading.jl:1342


HGramModel{pc, pc, 5}(Dict{Tuple{pc, Vector{pc}}, Int64}((pc('B'), [pc('A'), pc('G')]) => 1, (pc('B'), []) => 2, (pc('A'), [pc('B'), pc('A'), pc('G'), pc('G')]) => 1, (pc('G'), []) => 3, (pc('D'), []) => 1, (pc('A'), [pc('G'), pc('G'), pc('A'), pc('B')]) => 1, (pc('A'), [pc('G'), pc('G'), pc('A')]) => 1, (pc('B'), [pc('A')]) => 1, (pc('G'), [pc('G')]) => 1, (pc('D'), [pc('G')]) => 1…), Set(pc[pc('B'), pc('A'), pc('G'), pc('D')]))

### The model elements is a set containing the pitch classes encountered: 

In [11]:
m.elems

Set{pc} with 4 elements:
  pc('B')
  pc('A')
  pc('G')
  pc('D')

### The model database maps sequences to their number of occurences:

In [12]:
m.db

Dict{Tuple{pc, Vector{pc}}, Int64} with 29 entries:
  (pc('B'), [pc('A'), pc('G')])                   => 1
  (pc('B'), [])                                   => 2
  (pc('A'), [pc('B'), pc('A'), pc('G'), pc('G')]) => 1
  (pc('G'), [])                                   => 3
  (pc('D'), [])                                   => 1
  (pc('A'), [pc('G'), pc('G'), pc('A'), pc('B')]) => 1
  (pc('A'), [pc('G'), pc('G'), pc('A')])          => 1
  (pc('B'), [pc('A')])                            => 1
  (pc('G'), [pc('G')])                            => 1
  (pc('D'), [pc('G')])                            => 1
  (pc('G'), [pc('G'), pc('A')])                   => 1
  (pc('B'), [pc('D'), pc('G')])                   => 1
  (pc('B'), [pc('A'), pc('G'), pc('G')])          => 1
  (pc('G'), [pc('A')])                            => 1
  (pc('A'), [])                                   => 3
  (pc('A'), [pc('G')])                            => 1
  (pc('A'), [pc('B'), pc('D'), pc('G')])          => 1
  (pc('G'), [

# Maximum likelihood
The function ml computes the maximum likelihood of an element occuring in a given contexts from the model.

In [13]:
ml(pc('G'),m,m.elems)

0.3333333333333333

In [14]:
ml(pc('E'),m,m.elems)

0.0

In [15]:
ml(pc('G'),[pc('G')],m,m.elems)

0.3333333333333333

In [16]:
ml(pc('D'),[pc('B'),pc('A')],m,m.elems)

0.0

# Smoothed Prediction

The ppm function gives the probability of the symbol in a context from a given model and alphabet. The first two arguments specify the type of blending (backoff or interpolated) and the escape method (A, B, C, D or AX). 

## Backoff Smoothing

In [17]:
ppm(Backoff(),A(),pc('E'),[pc('E')],m,m.elems)

0.05

In [18]:
ppm(Backoff(),B(),pc('E'),[pc('E')],m,m.elems)

0.2

In [19]:
ppm(Backoff(),C(),pc('E'),[pc('E')],m,m.elems)

0.2

In [20]:
ppm(Backoff(),D(),pc('E'),m,m.elems)

0.2

In [21]:
ppm(Backoff(),AX(),pc('E'),m,m.elems)

0.2

## Interpolated Smoothing

In [22]:
ppm(Interpolated(),A(),pc('E'),[pc('E')],m,m.elems)

0.09999999999999998

In [23]:
ppm(Interpolated(),B(),pc('E'),[pc('E')],m,m.elems)

0.4444444444444444

In [24]:
ppm(Interpolated(),C(),pc('E'),[pc('G')],m,m.elems)

0.15384615384615385

In [25]:
ppm(Interpolated(),D(),pc('G'),[pc('G')],m,m.elems)

0.41666666666666663

In [26]:
ppm(Interpolated(),AX(),pc('G'),[pc('G')],m,m.elems)

0.36666666666666664

# Entropy
A Predictor contains all information required to make predictions: smoothing method, escape method, HGram model and alphabet. 

In [27]:
p = AtomicPredictor(Backoff(),B(),m)

AtomicPredictor{pc, pc}(Backoff(), B(), HGramModel{pc, pc, 5}(Dict{Tuple{pc, Vector{pc}}, Int64}((pc('B'), [pc('A'), pc('G')]) => 1, (pc('B'), []) => 2, (pc('A'), [pc('B'), pc('A'), pc('G'), pc('G')]) => 1, (pc('G'), []) => 3, (pc('D'), []) => 1, (pc('A'), [pc('G'), pc('G'), pc('A'), pc('B')]) => 1, (pc('A'), [pc('G'), pc('G'), pc('A')]) => 1, (pc('B'), [pc('A')]) => 1, (pc('G'), [pc('G')]) => 1, (pc('D'), [pc('G')]) => 1…), Set(pc[pc('B'), pc('A'), pc('G'), pc('D')])), Set(pc[pc('B'), pc('A'), pc('G'), pc('D')]))

A distribution contains a predictor and a context sequence. A distribution can be applied to an argument to obtain the probability. 

In [28]:
d = Distribution(p,[pc('E')])
d(pc('G'))

0.2222222222222222

In [29]:
entropy(d)

[3.1699250014423126, 2.1699250014423126, 2.1699250014423126, Inf]
[0.1111111111111111, 0.2222222222222222, 0.2222222222222222, 0.0]
NGram{pc, pc, 2}[NGram{pc, pc, 2}(pc('B'), pc[pc('E')]), NGram{pc, pc, 2}(pc('A'), pc[pc('E')]), NGram{pc, pc, 2}(pc('G'), pc[pc('E')]), NGram{pc, pc, 2}(pc('D'), pc[pc('E')])]


NaN

In [30]:
max_entropy(d)

2.0

In [31]:
information_content(pc('G'),d)

2.1699250014423126

In [32]:
redundancy(d)

[3.1699250014423126, 2.1699250014423126, 2.1699250014423126, Inf]
[0.1111111111111111, 0.2222222222222222, 0.2222222222222222, 0.0]
NGram{pc, pc, 2}[NGram{pc, pc, 2}(pc('B'), pc[pc('E')]), NGram{pc, pc, 2}(pc('A'), pc[pc('E')]), NGram{pc, pc, 2}(pc('G'), pc[pc('E')]), NGram{pc, pc, 2}(pc('D'), pc[pc('E')])]


NaN

In [33]:
mean_information_content([pc('G'),pc('G')],p)

0.1111111111111111